#### Scratch pad area

In [6]:
def counter(fn):
    count = 0

    """using `*args, **kwargs` means we can call any func `fn` with any
    combination of positional and keyword-only arguments
    """
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'function {fn.__name__} was called {count} times.')
        return fn(*args, **kwargs)

    return inner

In [7]:
def add(a, b=0):
    return a + b

In [8]:
add = counter(add)  # add is pointing to the above func

In [13]:
result = add(1, 2,)

function add was called 3 times.


In [14]:
result

3

#### So, how we relate this closure concepts to `decorator`?
##### We essentially modified our `add` function by wrapping it insikde another function that added some functionality to it

We also say that we **`decorated`** our function `add` with the function `counter`

So, what happens to `Decorators`
> - takes a function as an argument
> - return a closure
> - the closure usually accepts any combination of parameters
> - runs some code in the inner function (closure)
> - the closure function calls the original frunction using the arguments passed to it
> -0 returns whatever is returned by that function call

```python
my_func = func(my_func)

@counter
def add(a, b):
    return a + b

# is the same as writing

def add(a, b):
    return a + b

add = counter(add)

### `Introspecting` Decorated Functions

In [15]:
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'{fn.__name__} has called {count} times.')
        return fn(*args, **kwargs)
    return inner

In [16]:
@counter
def mult(a, b, c=1):
    """return the product of three values
    """
    return a * b * c

In [18]:
# it can be equally written as below

mult = counter(mult)

#### Let's look at the `name of property`

In [19]:
mult.__name__

'inner'

#### It's no longer the application name `mult` but the inner func name

#### `mult`'s name changed when we decorate it. They are not the same function after all

In [ ]:
help(mult)  # we've lost all info on original mult

Help on function inner in module __main__:

inner(*args, **kwargs)



#### now, this `inner` thing, does not help when we are debugging, because it says nothing => `we have lost our docstrings`

even uising `inspect` module's `signature` does not yield better results

#### The way to fix it!
##### We could try to fix this problem, at least for the docstring and function name as follows:

In [22]:
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Func {fn.__name__} was called {count} times.')
    
    # to fix with the two stmt below
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner

#### However, this does not solve all the problems, the signature is still not recovered

In [24]:
help(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



#### The `functools.wraps` function

##### the `functools` module has a `wraps` function that we can use to fix the metadata of our `inner` function in our decorator

In [25]:
from functools import wraps

#### In fact, the `wraps` func is itself a decorator

##### but it needs to know what was our "original" function - in this case `fn`

In [ ]:
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(count)
        return fn(*args, **kwargs)
    inner = wraps(fn)(inner)  # we call original func wraps(fn)
    return inner

#### above: We call wraps(fn) function -  fn, which is `original function)`, that is going to return a `func`, and that function now is going to take `inner` as its args, and that is going to return a `closure`, with which we are going to assign to inner. Then we return 

### It's easier to understand with the @wraps(fn) in the second code blocks\
```python
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(count)
        return fn(*args, **kwargs)
    
    inner = wraps(fn)(inner)
    
    retrurn inner

```

#### 2nd way 
```python
def counter(fn):
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(count)
        return fn(*args, **kwargs)
        
    return inner

In [ ]:
def counter(fn):
    count = 0
    @wraps(fn)  # the wrap needs to take fn as the arg in order to decorate it
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(count)
        return fn(*args, **kwargs)
    return inner

In [29]:
# now we can decorate it

@counter
def mult(a:int, b:int, c:int=1):
    """return the product of three numbers
    """
    return a * b * c

In [30]:
# now we call help to see what's changed?

help(mult)

Help on function mult in module __main__:

mult(a: int, b: int, c: int = 1)
    return the product of three numbers



#### And `introspection` using the `inspect` module works as expected:

```python
inspect.signature(mult)

In [32]:
inspect.signature(mult)

NameError: name 'inspect' is not defined

In [35]:
import inspect

inspect.signature(mult)

<Signature (a: int, b: int, c: int = 1)>

In [34]:
print(inspect.signature(mult))

(a: int, b: int, c: int = 1)


In [37]:
import inspect ## import insepct module

def mult2(x, y):
    return x * y

print(inspect.signature(mult2))
inspect.signature(mult2)

(x, y)


<Signature (x, y)>

### Decorators (Part 1)

Recall the example in  ths last section where we wrote a simple closure to count how many times a function had been run:

In [49]:
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

    return inner

In [50]:
def add(a, b=0):
    """
    returns the sum of a and b
    """
    return a + b

In [51]:
help(add)

Help on function add in module __main__:

add(a, b=0)
    returns the sum of a and b



Here's the memory address that add points to:

In [47]:
id(add)

4407120800

#### Aaron - let's introspect

In [53]:
# by Aaron Hung
import inspect
print(f'getsource for "{add.__name__}":\n{inspect.getsource(add)}')
print(f'getsource for "{counter.__name__}":\n{inspect.getsource(counter)}')
print(f'~~~~~')
print(f'signature for "{add.__name__}":\n{inspect.signature(add)}')
print(f'signature for "{counter.__name__}":\n{inspect.signature(counter)}')

getsource for "add":
def add(a, b=0):
    """
    returns the sum of a and b
    """
    return a + b

getsource for "counter":
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

    return inner

~~~~~
signature for "add":
(a, b=0)
signature for "counter":
(fn)


Now we create a closure using the add function as an argument to the counter function

In [54]:
add = counter(add)

And you'll notice that add is no longer the same function as before. 

Indeed the memory address add points to is no longer the same:

In [41]:
id(add)

4395070656

In [55]:
# by Aaron Hung
import inspect
print(f'getsource for "{add.__name__}":\n{inspect.getsource(add)}')
print(f'getsource for "{counter.__name__}":\n{inspect.getsource(counter)}')
print(f'~~~~~')
print(f'signature for "{add.__name__}":\n{inspect.signature(add)}')
print(f'signature for "{counter.__name__}":\n{inspect.signature(counter)}')

getsource for "inner":
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

getsource for "counter":
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

    return inner

~~~~~
signature for "inner":
(*args, **kwargs)
signature for "counter":
(fn)


### The above `add`'s source and signature has become `inner` - the `wrapper of decorator`

In [56]:
add(1, 2)

Function add was called 1 times


3

In [57]:
add(2, 2)

Function add was called 2 times


4

What happened is that we put our add function 'through' the counter function - we usually say that we decorated our function add.

And we call that counter function a decorator.

There is a shorthand way of decorating our function without having to type:

`func = counter(func)`

In [61]:
del add

In [62]:
# by Aaron Hung
import inspect
print(f'getsource for "{add.__name__}":\n{inspect.getsource(add)}')
print(f'getsource for "{counter.__name__}":\n{inspect.getsource(counter)}')
print(f'~~~~~')
print(f'signature for "{add.__name__}":\n{inspect.signature(add)}')
print(f'signature for "{counter.__name__}":\n{inspect.signature(counter)}')

NameError: name 'add' is not defined

In [63]:
@counter
def add(a: float, b: float) -> float:
    """
    return the sum of a and b
    """
    return a + b

In [64]:
# by Aaron Hung
import inspect
print(f'getsource for "{add.__name__}":\n{inspect.getsource(add)}')
print(f'getsource for "{counter.__name__}":\n{inspect.getsource(counter)}')
print(f'~~~~~')
print(f'signature for "{add.__name__}":\n{inspect.signature(add)}')
print(f'signature for "{counter.__name__}":\n{inspect.signature(counter)}')

getsource for "inner":
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

getsource for "counter":
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

    return inner

~~~~~
signature for "inner":
(*args, **kwargs)
signature for "counter":
(fn)


In [65]:
add(1, 2)

Function add was called 1 times


3

In [66]:
add(3, 4)

Function add was called 2 times


7

In [67]:
@counter
def mult(a: float, b: float, c: float=1) -> float:
    """
    retrurn the product of a, b, and c
    """
    return a * b * c

In [21]:
mult(1, 2, 3)

Function mult was called 1 times


6

In [22]:
mult(2, 2, 3)

Function mult was called 2 times


12

Let's do a little bit of introspection on our two decorated functions:

In [23]:
add.__name__

'inner'

In [24]:
mult.__name__

'inner'

In [28]:
add.__code__.co_freevars, mult.__code__.co_freevars

(('count', 'fn'), ('count', 'fn'))

As you can see, the name of the function is no longer add or mult, but instead it is the name of that inner function in our decorator.

In [29]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [30]:
help(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



As you can see, we've also lost our docstring and parameter annotations!

### Introspecting the parameters of add and mult:

In [31]:
import inspect

In [32]:
inspect.getsource(add)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print(f'Function {fn.__name__} was called {count} times')\n        return fn(*args, **kwargs)\n"

In [33]:
inspect.getsource(mult)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print(f'Function {fn.__name__} was called {count} times')\n        return fn(*args, **kwargs)\n"

#### Even the signature is gone:

In [34]:
inspect.signature(add)

<Signature (*args, **kwargs)>

In [35]:
inspect.signature(mult)

<Signature (*args, **kwargs)>

### Aaron's comments with introspect

In [68]:
# by Aaron Hung
import inspect
print(f'getsource for "{add.__name__}":\n{inspect.getsource(add)}')
print(f'getsource for "{counter.__name__}":\n{inspect.getsource(counter)}')
print(f'~~~~~')
print(f'signature for "{add.__name__}":\n{inspect.signature(add)}')
print(f'signature for "{counter.__name__}":\n{inspect.signature(counter)}') 

getsource for "inner":
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

getsource for "counter":
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

    return inner

~~~~~
signature for "inner":
(*args, **kwargs)
signature for "counter":
(fn)


In [69]:
# introspect mult - by Aaron Hung
import inspect
print(f'getsource for "{mult.__name__}":\n{inspect.getsource(mult)}')
print(f'getsource for "{counter.__name__}":\n{inspect.getsource(counter)}')
print(f'~~~~~')
print(f'signature for "{mult.__name__}":\n{inspect.signature(mult)}')
print(f'signature for "{counter.__name__}":\n{inspect.signature(counter)}') 

getsource for "inner":
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

getsource for "counter":
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

    return inner

~~~~~
signature for "inner":
(*args, **kwargs)
signature for "counter":
(fn)


In [70]:
print(inspect.getsource(add))
print(inspect.getsource(mult))
print(inspect.signature(add))
print(inspect.signature(mult))

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args, **kwargs)

(*args, **kwargs)
(*args, **kwargs)


##### In general, when  we create decorated functions, we end up "losing" a lot of the metadata of our original function

##### However, we can put that information back in - it can get quite complicated.

Let's see how we might be able to do that for some simple things, like the docstring and the function name

In [72]:
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"{fn.__name__} was called {count} times")
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    
    return inner


In [73]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns the sum of two integers (note: fixed mis-info by decoratting in the docstring)
    """
    return a + b

In [74]:
help(add)

Help on function add in module __main__:

add(*args, **kwargs)
    returns the sum of two integers (note: fixed mis-info by decoratting in the docstring)



In [75]:
add.__name__

'add'

##### At least we have the docstring and function name back...

#### But what about the parameters?
> - Our real add function takes two positional parameters, but because the closure used to generic way of accepting *args and **kwargs, we lose this information
>
##### We can use a special function in the `functools` module, called `wraps`. In fact, that function is a decorator itself!

In [76]:
from functools import wraps

In [77]:
def counter(fn): 
    count = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"{fn.__name__} was called {count} times")
        return fn(*args, **kwargs)

    return inner

In [78]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns the sum of two integers (note: fixed info messed by decoratting in the docstring)
    """
    return a + b

In [79]:
help(add)

Help on function add in module __main__:

add(a: int, b: int = 10) -> int
    returns the sum of two integers (note: fixed info messed by decoratting in the docstring)



##### Yes! Everything is back to normal

In [82]:
inspect.getsource(add)

'@counter\ndef add(a: int, b: int=10) -> int:\n    """\n    returns the sum of two integers (note: fixed info messed by decoratting in the docstring)\n    """\n    return a + b\n'

In [81]:
print(inspect.getsource(add))

@counter
def add(a: int, b: int=10) -> int:
    """
    returns the sum of two integers (note: fixed info messed by decoratting in the docstring)
    """
    return a + b



In [88]:
inspect.signature(add)

<Signature (a: int, b: int = 10) -> int>

In [85]:
inspect.signature(add).parameters

mappingproxy({'a': <Parameter "a: int">, 'b': <Parameter "b: int = 10">})

#### > Aaron's Experiments on `Decorator Part1`

Recall the example in the last section where we wrote a simple closure to count how many times a function had been run:

In [2]:
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"Function {fn.__name__} was called {count} times.")
        return fn(*args, **kwargs)
    return inner

In [5]:
def add(a, b=0):
    """returns the sum of a and b
    """
    return a + b

In [6]:
help(add)

Help on function add in module __main__:

add(a, b=0)
    returns the sum of a and b



Here's the memory address that `add` points to:

In [7]:
id(add)

4439839968

Now we create a closure using the `add` function as an ar4gument to the `counter` function:

In [8]:
add = counter(add)

And you'll note that `add` is no longer the same function as before. Indeed the memory address `add` points to is no longer the same:

In [9]:
id(add)

4439847168

In [10]:
add(1, 2)

Function add was called 1 times.


3

In [11]:
add(2, 2)

Function add was called 2 times.


4

What happened is that we put our add function 'through' the counter function - we usually say that we decorated our function add.

And we call that counter function a `decorator`.

There is a shorthand way of decorating our function without having to type:

`func = counter(func)`

In [12]:
@counter
def mult(a: float, b: float=1, c: float=1) -> float:
    """
    returns the product of a, b, and c
    """
    return a * b * c

In [13]:
mult(1, 2, 3)

Function mult was called 1 times.


6

In [14]:
mult(2, 2, 2)

Function mult was called 2 times.


8

Let's do a little bit of introspection on our two decorated functions:

In [15]:
add.__name__

'inner'

In [16]:
mult.__name__

'inner'

As you can see, the nmame of the function is no longer add or mult, but instead it is the name of that inner function in our decorator.

In [17]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [18]:
help(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



As you can see, we've also lost our docstring and parameter annotations!

What about introspecint the parameters of add and mult:

In [19]:
import inspect

In [20]:
inspect.getsource(add)

'    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print(f"Function {fn.__name__} was called {count} times.")\n        return fn(*args, **kwargs)\n'

In [21]:
inspect.getsource(mult)

'    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print(f"Function {fn.__name__} was called {count} times.")\n        return fn(*args, **kwargs)\n'

Even the signature is gone:

In [22]:
inspect.signature(add)

<Signature (*args, **kwargs)>

In [23]:
inspect.signature(mult)

<Signature (*args, **kwargs)>

Even the parameter defaults documentation is are gone:

In [24]:
inspect.signature(add).parameters

mappingproxy({'args': <Parameter "*args">, 'kwargs': <Parameter "**kwargs">})

In general, when we careate decorated functions, we end up "losing" a lot of the metadata of our original function!

However, we can put that information back in - it can get quite complicated. 

Let's see how we might be able to do that for some simple things, like the docstring and the function name.

In [25]:
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"{fn.__name__} was called {count} times.")
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner


In [26]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns some of two integers

    Args:
        a (int): _description_
        b (int, optional): _description_. Defaults to 10.

    Returns:
        int: _description_
    """
    return a + b

In [27]:
help(add)

Help on function add in module __main__:

add(*args, **kwargs)
    returns some of two integers
    
    Args:
        a (int): _description_
        b (int, optional): _description_. Defaults to 10.
    
    Returns:
        int: _description_



In [28]:
add.__name__

'add'

At least we have the docstring and function name back... But what about the paramters?

Our real add function takes two positional parameters, but because the closure used a generic way of accepting `*args` and `**kwargs`, we lose this information

We can use a special function in the functools module, called wraps. In tact that function is a decorator itself!

In [29]:
from functools import wraps

In [30]:
def counter(fn):
    count = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count 
        count += 1
        print(f"{fn.__name__} was called {count} times.")
        return fn(*args, **kwargs)
    return inner

In [31]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returnss sum of two integers

    """
    return a + b

In [32]:
help(add)

Help on function add in module __main__:

add(a: int, b: int = 10) -> int
    returnss sum of two integers



`Yeah! Everything is back to normal!`

In [33]:
inspect.getsource(add)

'@counter\ndef add(a: int, b: int=10) -> int:\n    """\n    returnss sum of two integers\n\n    """\n    return a + b\n'

In [34]:
inspect.getsource(add)

'@counter\ndef add(a: int, b: int=10) -> int:\n    """\n    returnss sum of two integers\n\n    """\n    return a + b\n'

In [35]:
inspect.signature(add)

<Signature (a: int, b: int = 10) -> int>

In [36]:
inspect.signature(add).parameters

mappingproxy({'a': <Parameter "a: int">, 'b': <Parameter "b: int = 10">})

Recall the example in the last section where we wrote a simple closure to count how many times a function had been run:

In [1]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [2]:
def add(a, b=0):
    """
    returns the sum of a and b
    """
    return a + b

In [3]:
help(add)

Help on function add in module __main__:

add(a, b=0)
    returns the sum of a and b



Here's the memory address that `add` points to:

In [4]:
id(add)

2352389334696

Now we create a closure using the `add` function as an argument to the `counter` function:

In [5]:
add = counter(add)

And you'll note that `add` is no longer the same function as before. Indeed the memory address `add` points to is no longer the same:

In [6]:
id(add)

2352404346128

In [7]:
add(1, 2)

Function add was called 1 times


3

In [8]:
add(2, 2)

Function add was called 2 times


4

What happened is that we put our **add** function 'through' the **counter** function - we usually say that we **decorated** our function **add**.

And we call that **counter** function a **decorator**.

There is a shorthand way of decorating our function without having to type:

``func = counter(func)``

In [9]:
@counter
def mult(a: float, b: float=1, c: float=1) -> float:
    """
    returns the product of a, b, and c
    """
    return a * b * c

In [10]:
mult(1, 2, 3)

Function mult was called 1 times


6

In [11]:
mult(2, 2, 2)

Function mult was called 2 times


8

Let's do a little bit of introspection on our two decorated functions:

In [12]:
add.__name__

'inner'

In [13]:
mult.__name__

'inner'

As you can see, the name of the function is no longer **add** or **mult**, but instead it is the name of that **inner** function in our decorator.

In [14]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [15]:
help(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



As you can see, we've also lost our docstring and parameter annotations!

What about introspecting the parameters of **add** and **mult**:

In [16]:
import inspect

In [17]:
inspect.getsource(add)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

In [18]:
inspect.getsource(mult)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

Even the signature is gone:

In [19]:
inspect.signature(add)

<Signature (*args, **kwargs)>

In [20]:
inspect.signature(mult)

<Signature (*args, **kwargs)>

Even the parameter defaults documentation is are gone:

In [21]:
inspect.signature(add).parameters

mappingproxy({'args': <Parameter "*args">, 'kwargs': <Parameter "**kwargs">})

In general, when we create decorated functions, we end up "losing" a lot of the metadata of our original function!

However, we **can** put that information back in - it can get quite complicated.

Let's see how we might be able to do that for some simple things, like the docstring and the function name.

In [22]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner

In [23]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [24]:
help(add)

Help on function add in module __main__:

add(*args, **kwargs)
    returns sum of two integers



In [25]:
add.__name__

'add'

At least we have the docstring and function name back... But what about the parameters? Our real **add** function takes two positional parameters, but because the closure used a generic way of accepting **\*args** and **\*\*kwargs**, we lose this information

We can use a special function in the **functools** module, called **wraps**. In fact, that function is a decorator itself!

In [26]:
from functools import wraps

In [27]:
def counter(fn):
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))
        return fn(*args, **kwargs)

    return inner

In [28]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [29]:
help(add)

Help on function add in module __main__:

add(a:int, b:int=10) -> int
    returns sum of two integers



Yay!!! Everything is back to normal.

In [30]:
inspect.getsource(add)

'@counter\ndef add(a: int, b: int=10) -> int:\n    """\n    returns sum of two integers\n    """\n    return a + b\n'

In [31]:
inspect.signature(add)

<Signature (a:int, b:int=10) -> int>

In [32]:
inspect.signature(add).parameters

mappingproxy({'a': <Parameter "a:int">, 'b': <Parameter "b:int=10">})